* [Rich feature hierarchies for accurate object detection and semantic segmentation](https://arxiv.org/pdf/1311.2524)
* [Fast R-CNN](https://arxiv.org/abs/1504.08083)
* [Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks](https://arxiv.org/abs/1506.01497)

# R-CNN
![](https://cdn-images-1.medium.com/max/1600/1*REPHY47zAyzgbNKC6zlvBQ.png)

* 用现成的Selective Search获取大约2000个候选框, 然后每个框warp后用CNN提取特征, 最后SVM进行分类, 同时会预测四个offset来提高bbox的准确性。每个候选框都要提取特征, 这是非常慢的，能不能只提取一次特征?

# Fast R-CNN
![](https://cdn-images-1.medium.com/max/1600/1*0pMP3aY8blSpva5tvWbnKA.png)

* 只提取一次全局特征, 然后用候选框来提取特征, 用**Roi pooling**成固定尺寸, 然后送进全连接层做分类和回归。现在的瓶颈就是region proposals了


# Faster R-CNN
![](https://lilianweng.github.io/lil-log/assets/images/faster-RCNN.png)

* Faster R-CNN 主要有三部分组成, 第一个就是backbone来提取特征, 第二个是RPN(Region proposal network)得到一定数量的ROIs, 然后将ROIs和特征输入到ROIHead, 进行分类和回归。重点在于第二和第三部分。

![](https://pic3.zhimg.com/80/v2-4e372e4536ef6d3d28ebd8803a9b13e2_hd.jpg)

### RPN

1. anchor
    * 每个特征图上的点都对应9个anchor
2. 具体结构
    ![](https://pic3.zhimg.com/80/v2-e7eeb94a86ece2dadfa9db2277f7d016_hd.jpg)
    * AnchorTargetCreator就是将这么多anchor选出256个来进行分类和回归位置
        * 和gt的iou超过0.7或最高的anchor作为正样本
        * 和gt的iou小于0.3的anchor作为负样本
    * gt_loc则是由4个位置参数(tx,ty,tw,th)组成, 损失用的是Smooth_l1_loss, 只计算正样本
        ![](https://www.zhihu.com/equation?tex=t_x+%3D+%28x+%E2%88%92+x_a%29%2Fw_a%3B+t_y+%3D+%28y+%E2%88%92+y_a%29%2Fh_a%3B%5C%5C+t_w+%3D+log%28w%2Fw_a%29%3B+t_h+%3D+log%28h%2Fh_a%29%3B%5C%5C+t_x%5E%2A+%3D+%28x%5E%2A+%E2%88%92+x_a%29%2Fw_a%3B+t_y%5E%2A+%3D+%28y%5E%2A+%E2%88%92+y_a%29%2Fh_a%3B%5C%5C+t_w%5E%2A+%3D+log%28w%5E%2A%2Fw_a%29%3B+t_h%5E%2A+%3D+log%28h%5E%2A%2Fh_a%29%3B%5C%5C)
         * scale-invariant transformatio
3. RPN生成RoIs
    * RPN不仅自身训练,还提供ROIs给Fast RCNN训练.
    * 根据 foreground score选出 12000个anchor, 利用回归的位置参数, 修正这些anchor的位置
    * 采用NMS, 选出概率最大的2000个ROIs
    * 此操作不需要反向传播
    
 

### ROIHead/Fast R-CNN

* 从2000个ROis选出128个sample rois, 然后使用ROIPooling将这些区域pooling到同一尺度。
* RoI Pooling 是一种特殊的Pooling操作，给定一张图片的Feature map (512×H/16×W/16) ，和128个候选区域的坐标（128×4），RoI Pooling将这些区域统一下采样到 （512×7×7），就得到了128×512×7×7的向量。可以看成是一个batch-size=128，通道数为512，7×7的feature map。
* 训练
    * RoIs和gt_bboxes 的IoU大于0.5的，选择一些（比如32个）
    * 选择 RoIs和gt_bboxes的IoU小于等于0（或者0.1）的选择一些（比如 128-32=96个）作为负样本

# ROiAlign
The RoIAlign layer is designed to fix the location misalignment caused by quantization in the RoI pooling. RoIAlign removes the hash quantization, for example, by using x/16 instead of [x/16], so that the extracted features can be properly aligned with the input pixels. Bilinear interpolation is used for computing the floating-point location values in the input.
![](https://lilianweng.github.io/lil-log/assets/images/roi-align.png)

![](https://lilianweng.github.io/lil-log/assets/images/rcnn-family-summary.png)